<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat

In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [4]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [5]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.13.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();


Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.13.0

Using Azure OpenAI Service


### Step 2 - Execute the Chat Completion Service with Prompt Execution Settings

Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [6]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var simpleDecisionPrompt = "Provide some decision frameworks that can help improve the quality of decisions.";
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Certainly! Decision-making frameworks can significantly enhance the quality of decisions by providing structured approaches to evaluate options and outcomes. Here are some widely recognized frameworks:

### 1. **SWOT Analysis**
   - **Strengths**: Identify internal strengths.
   - **Weaknesses**: Identify internal weaknesses.
   - **Opportunities**: Identify external opportunities.
   - **Threats**: Identify external threats.
   - **Usage**: Helps in strategic planning and understanding the internal and external factors affecting a decision.

### 2. **PEST Analysis**
   - **Political**: Evaluate political factors.
   - **Economic**: Evaluate economic factors.
   - **Social**: Evaluate social factors.
   - **Technological**: Evaluate technological factors.
   - **Usage**: Useful for understanding the macro-environmental factors that could impact a decision.

### 3. **Decision Matrix (Weighted Scoring Model)**
   - **Criteria Identification**: List the criteria important for the decision

### Step 3 - Execute a Semantic Kernel Chat Completion with Streaming

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [7]:
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Certainly! Improving the quality of decisions can be achieved through various decision-making frameworks. These frameworks provide structured approaches that ensure all necessary factors are considered. Here are some well-regarded frameworks:

### 1. **Rational Decision-Making Model**
This is a step-by-step approach for making logically sound decisions.

Steps include:
- **Define the problem**
- **Identify decision criteria**
- **Weight the criteria**
- **Generate alternatives**
- **Rate each alternative on each criterion**
- **Compute the optimal decision**

### 2. **Kepner-Tregoe Matrix**
A systematic process for problem-solving and decision-making.

Steps include:
- **Situational Analysis**: Clarify the situation, define problems, and prioritize
- **Problem Analysis**: Determine root causes
- **Decision Analysis**: Define objectives, evaluate alternatives against criteria
- **Potential Problem Analysis**: Identify potential problems and preventive actions

### 3. **SWOT Analysis**
A

### Step 4 - Execute a Semantic Kernel with Chat History

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt
* User Message - A message from the user
* Assistante Message - A message from the LLM

This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [5]:
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var simpleDecisionPrompt = "Provide five decision frameworks that can help improve the quality of decisions.";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Decision-making is a critical skill in various aspects of life and work, and having robust frameworks can significantly enhance the quality and outcomes of the decisions. Here are five decision frameworks that can aid in this process:

1. **SWOT Analysis**:
   - **Description**: SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework helps in understanding the internal and external factors that could impact the decision. It's particularly useful in strategic decision-making.
   - **Application**: Use this when entering a new market, launching a product, or when decision outcomes can be influenced by external and internal factors. It ensures that you are aware of your capabilities and the environment in which you are operating.
   - **Benefits**: Provides a balanced overview of both positive and negative factors and helps in identifying the key elements that could influence the success or failure of a decision.

2. **Decision Matrix (Weighted Scoring Model)**:


### Step 5 - Using Semantic Kernel Chat History in Conversation

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have the proper context to answer the question.

In [6]:
var simpleDecisionPromptFollowup = "Which of these 5 decision frameworks would work best for making a decision to purchase a car. Select only the single best one.";

chatHistory.AddUserMessage(simpleDecisionPromptFollowup);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

For making a decision to purchase a car, the **Decision Matrix (Weighted Scoring Model)** is the single best framework among the options provided. This approach stands out because purchasing a car involves evaluating multiple factors, each of which may vary in importance according to your personal needs and preferences. 

### How to Apply the Decision Matrix to Car Buying:

1. **Identify Criteria**: List all factors that are important to you in a car. Common criteria might include fuel efficiency, cost, reliability, brand, design, safety features, resale value, service availability, etc.

2. **Weight the Criteria**: Assign a weight to each criterion based on how important it is to you. For example, if safety is a top priority, you might give it a higher weight compared to other less critical features.

3. **List the Options**: Write down the car models you are considering. 

4. **Rate Each Option**: Score each car model against each criterion on a consistent scale (e.g., 1-10). For ins

In [7]:
var simpleDecisionPromptFollowupPartTwo = "Which of these 5 decision frameworks listed above are used by the military intelligence community?";

chatHistory.AddUserMessage(simpleDecisionPromptFollowupPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

The military intelligence community frequently employs decision-making frameworks to analyze data and make strategic decisions efficiently and effectively. Among the five frameworks previously listed, two are particularly pertinent to military operations:

1. **SWOT Analysis**:
   - **Usage in Military Intelligence**: The SWOT framework is utilized to evaluate the strengths, weaknesses, opportunities, and threats concerning specific operations, strategic positions, or in broader strategic planning. In the military context, strengths and weaknesses often pertain to the capabilities and limitations of own forces, while opportunities and threats typically focus on the operational environment or the enemy’s capabilities.
   - **Benefits**: This analysis helps the military to exploit advantages, shore up weaknesses, capitalize on strategic opportunities, and mitigate external threats. It fosters a comprehensive view that is crucial for operation planning and national security strategy formu

### Step 6 - Semantic Kernel inspecting Chat History 

The Semantic Kernel Chat History is a transparent object that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [8]:
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 7
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five decision frameworks that can help improve the quality of decisions.
# assistant: Decision-making is a critical skill in various aspects of life and work, and having robust frameworks can significantly enhance the quality and outcomes of the decisions. Here are five decision frameworks that can aid in this process:

1. **SWOT Analysis**:
   - **Description**: SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework helps in understanding the internal and external factors that could impact the decision. It's particularly useful in strategic decision-making.
   - **Application**: Use this when entering a new market, launching a product, or when decision outcomes can be in